In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

In [ ]:
def load(base_dir='./animals'):
    data, labels = [], []
    categories = os.listdir(base_dir)
    paths = [os.path.join(base_dir, category) for category in categories]
    for path in paths:
        files = os.listdir(path)
        filenames = [os.path.join(path, file) for file in files]
        for filename in filenames:
            img = cv2.imread(filename)
            image = cv2.resize(img, (32, 32), interpolation=cv2.INTER_AREA)
            data.append(image.astype('float') / 255.0)
            labels.append(files[0].split('_')[0])
    data = np.array(data)
    labels = np.array(labels)
    return (data, labels, categories)

In [ ]:
def CNN(data, labels, categories, learning_rates=[0.005], epochs=50):
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.32, random_state=42)
    Y_train = LabelBinarizer().fit_transform(y_train)
    Y_test = LabelBinarizer().fit_transform(y_test)
    models = []
    results = []
    for lr in learning_rates:
        model = Sequential()
        
        # Model 1
        model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(len(categories)))
        model.add(Activation('softmax'))
        
        # Training & Testing
        model.compile(
            optimizer=SGD(lr=lr),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        H = model.fit(X_train, Y_train, epochs=epochs, batch_size=32, validation_split=0.25)
        predictions = model.predict(X_test)
        # predicted_labels = [categories[i] for i in predictions.argmax(axis=1)]
        # print(*zip(predicted_labels, y_test))     

        # Store results
        models.append((epochs, H))
        results.append((
            lr,
            classification_report(
                Y_test.argmax(axis=1),
                predictions.argmax(axis=1),
                target_names=categories
            )
        ))
    return (models, results)

In [ ]:
def plot(models):
    for n, H in models:
        plt.style.use('ggplot')
        plt.figure()
        plt.plot(np.arange(0, n), H.history['loss'], label='train_loss')
        plt.plot(np.arange(0, n), H.history['val_loss'], label='val_loss')
        plt.plot(np.arange(0, n), H.history['accuracy'], label='train_acc')
        plt.plot(np.arange(0, n), H.history['val_accuracy'], label='val_acc')
        plt.title('Training Loss and Accuracy')
        plt.xlabel('Epoch #')
        plt.ylabel('Loss/Accuracy')
        plt.legend()
        plt.show()

In [ ]:
base_dir = '/Users/libao/Documents/data/animals'
data, labels, categories = load(base_dir=base_dir)
models, results = CNN(data, labels, categories)
plot(models)
for result in results:
    print('Learning rate: {:f}'.format(result[0]))
    print(result[1])